In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import requests
from requests_oauthlib import OAuth1
import argparse
import json
from time import gmtime, strftime
import os
import glob
import time

In [2]:
listings = pd.read_csv(r'data\boston-airbnb-open-data\listings.csv')

neighbourhood = pd.read_csv(r'data\boston-airbnb-open-data\neighbourhoods.csv')

print(listings.head())
print(neighbourhood.head())

         id                            listing_url     scrape_id last_scraped  \
0  12147973  https://www.airbnb.com/rooms/12147973  2.016090e+13     9/7/2016   
1   3075044   https://www.airbnb.com/rooms/3075044  2.016090e+13     9/7/2016   
2      6976      https://www.airbnb.com/rooms/6976  2.016090e+13     9/7/2016   
3   1436513   https://www.airbnb.com/rooms/1436513  2.016090e+13     9/7/2016   
4   7651065   https://www.airbnb.com/rooms/7651065  2.016090e+13     9/7/2016   

                                            name  \
0                     Sunny Bungalow in the City   
1              Charming room in pet friendly apt   
2               Mexican Folk Art Haven in Boston   
3  Spacious Sunny Bedroom Suite in Historic Home   
4                            Come Home to Boston   

                                             summary  \
0  Cozy, sunny, family home.  Master bedroom high...   
1  Charming and quiet room in a second floor 1910...   
2  Come stay with a friendly, mi

# Data Scrubbing

In [14]:
print("Shape of this dataframe.{val}".format(val=listings.shape))

pd.DataFrame(listings.columns)

unique_in_availability = listings.has_availability.unique()

print(unique_in_availability)

listings = listings.drop(['has_availability'], axis=1)
print("Shape after dropping column.{val}".format(val=listings.shape))

Shape of this dataframe.(3585, 95)
[ nan]
Shape after dropping column.(3585, 94)


In [16]:
unique_in_license = listings.license.unique()
unique_in_jurisdiction = listings.jurisdiction_names.unique()

print(unique_in_license)
print(unique_in_jurisdiction)
listings = listings.drop(['license','jurisdiction_names'], axis=1)
print("Shape after dropping column.{val}".format(val=listings.shape))

[ nan]
[ nan]
Shape after dropping column.(3585, 92)


In [18]:
unique_in_experience = listings.experiences_offered.unique()
print(unique_in_experience)

listings = listings.drop(['experiences_offered'], axis=1)

print("Shape after dropping column.{val}".format(val=listings.shape))

['none']
Shape after dropping column.(3585, 91)


In [21]:
unique_in_scrape_id = listings.scrape_id.unique()
print(unique_in_scrape_id)

listings = listings.drop(['scrape_id'], axis=1)

print("Shape after dropping column.{val}".format(val=listings.shape))

[  2.01609000e+13]
Shape after dropping column.(3585, 90)


In [ ]:
#Filling the missing values appropriately

In [27]:
mean_ratings = np.mean(listings['review_scores_rating'])

print(mean_ratings)

listings['review_scores_rating'] = listings.review_scores_rating.fillna(mean_ratings)

#print(listings['review_scores_rating'])

91.9166666667


In [20]:
#replacing neighbourhood from 'listings' dataframe to correct value from 'neighbourhood' dataframe

#listings.host_neighbourhood = listings.host_neighbourhood.astype(str)

neighbourhood.neighbourhood = neighbourhood.neighbourhood.astype(str)
listings.host_neighbourhood = listings.host_neighbourhood.fillna("Not Specified")
listings.neighborhood_overview = listings.neighborhood_overview.astype(str)
        
print(listings.host_neighbourhood)

0          Roslindale
1          Roslindale
2          Roslindale
3       Not Specified
4          Roslindale
5          Roslindale
6          Roslindale
7          Roslindale
8          Roslindale
9          Roslindale
10         Roslindale
11         Roslindale
12         Roslindale
13         Roslindale
14         Roslindale
15         Roslindale
16         Roslindale
17         Roslindale
18         Roslindale
19         Roslindale
20         Roslindale
21         Roslindale
22         Roslindale
23      Not Specified
24         Roslindale
25         Roslindale
26         Roslindale
27         Roslindale
28         Roslindale
29         Roslindale
            ...      
3555        Brookline
3556       Somerville
3557    Not Specified
3558    Not Specified
3559    Not Specified
3560       Somerville
3561    Not Specified
3562    Not Specified
3563       Somerville
3564       Somerville
3565    Not Specified
3566        Brookline
3567      Gay Village
3568       Somerville
3569    No

# Fetching Foursquare's Data

In [36]:
from __future__ import unicode_literals
import requests
import argparse
import json
import os
from time import gmtime, strftime
import time
import os


searchUrl = 'https://api.foursquare.com/v2/venues/search?ll=42.36,-71.05&oauth_token=YOUR_AUTH_TOKEN&limit=50'

r = requests.get(searchUrl, stream = True)

data = dict(r.json())

fname = 'foursquare.json'

rec = dict(r.json())
json.dumps(rec)

with open(fname ,'w')as f:
    json.dump(rec, f)

print("Successfully dumped data to json file.")

Successfully dumped data to json file.


# Fetching Twitter's Data

In [25]:
API_KEY = 'YOUR_API_KEY'
API_SECRET = 'YOUR_SECRET_KEY'
ACCESS_TOKEN = 'YOUR_ACCESS_TOKEN'
ACCESS_TOKEN_SECRET = 'YOUR_SECRET_ACCESS_TOKEN'

auth = OAuth1(API_KEY, API_SECRET, ACCESS_TOKEN, ACCESS_TOKEN_SECRET)

#requests.get(url, auth=auth)


geoloc = "42.36,-71.05,20mi"
searchUrl = "https://api.twitter.com/1.1/search/tweets.json?q=airbnb%20-from%3Astylishrentals&src=typd&lang=en&result_type=mix&count=1500&geocode="+geoloc;

tweets = requests.get(searchUrl, auth=auth, stream = True)

#print(type(r))
#for tweet in r.json():
    # print (tweet)
                
file_name = strftime("%Y-%m-%d-%H-%M-%S", gmtime())
file_name = "data/twitter/"+file_name+".json"

record = dict(tweets.json())
json.dumps(record)
with open(file_name ,'w')as f:
    json.dump(record, f)
twitter_filedata = json.load(open(file_name))
print(twitter_filedata)

{'statuses': [{'truncated': False, 'place': None, 'is_quote_status': False, 'user': {'entities': {'description': {'urls': []}, 'url': {'urls': [{'indices': [0, 22], 'url': 'http://t.co/hAW6TMt6eo', 'display_url': 'avacationrental4me.com', 'expanded_url': 'http://www.avacationrental4me.com'}]}}, 'is_translation_enabled': False, 'friends_count': 2680, 'description': 'We do not charge booking fees! Helping vacationers find and rent vacation rental homes in the USA, Europe, and more. Owners list your vacation rental here.', 'time_zone': 'Eastern Time (US & Canada)', 'profile_background_image_url_https': 'https://pbs.twimg.com/profile_background_images/856681090/6d4e1d21f0ea6dbaf8a672418398f071.jpeg', 'id': 86996695, 'profile_text_color': '333333', 'translator_type': 'none', 'lang': 'en', 'location': 'Boston,  USA', 'screen_name': 'avr4m', 'profile_image_url_https': 'https://pbs.twimg.com/profile_images/459029340634636288/fF1nFxyj_normal.jpeg', 'created_at': 'Mon Nov 02 18:23:35 +0000 2009'

### Leveraging twitter's pagination to get value from next 10 pages

In [26]:
max_count = 100
counter = 1

In [28]:
def getNextResults(twitter_filedata):
    return twitter_filedata['search_metadata']['next_results']

In [33]:
for i in range(1,3):
    toAppend = getNextResults(twitter_filedata)
    newURL = "https://api.twitter.com/1.1/search/tweets.json"+toAppend

    tweets = requests.get(searchUrl, auth=auth, stream = True)
    file_name = strftime("%Y-%m-%d-%H-%M-%S", gmtime())
    file_name = "data/twitter/"+file_name+".json"

    record = dict(tweets.json())
    json.dumps(record)
    with open(file_name ,'w')as f:
        json.dump(record, f)
    twitter_filedata = json.load(open(file_name))
    
    time.sleep(3)